In [5]:
import os
import copy
import time
import torch
import skimage
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform

torch.cuda.set_device(2)
print("complete")

complete


In [6]:
class Bird_Dataset_Local(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, train_file, root_dir, local_file, transform=None):
        f = open(train_file, 'r')
        self.train_list = f.readlines()
        f.close()
        self.vectors_list = np.load(local_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.train_list)

    def __getitem__(self, idx):
        line = self.train_list[idx]
        img_dir_label = line.strip('\n').split(' ')
        img_dir = os.path.join(self.root_dir, img_dir_label[0])
        image = io.imread(img_dir)
        if len(image.shape) == 2:
            image = image[:,:,np.newaxis]
            image = np.tile(image, [1, 1, 3])
        
        vectors = self.vectors_list[idx]
        N = vectors.shape[0]
        

        x1, x2, y1, y2 = np.min(vectors[:, 0]), np.max(vectors[:, 0]), \
                        np.min(vectors[:, 1]), np.max(vectors[:, 1])

        edge = 16
        m1, m2, n1, n2 = int(np.maximum(x1 * 16 + 8 - edge, 0)), int(np.minimum(x2 * 16 + 8 + edge, 448)), \
                        int(np.maximum(y1 * 16 + 8 - edge, 0)), int(np.minimum(y2 * 16 + 8 + edge, 448))
        
        image = skimage.util.img_as_ubyte(transform.resize(image, (448, 448)))
        image = image[m1:m2, n1:n2]

        label = int(img_dir_label[1])
        sample = {'image': image, 'label': label}
        
        if self.transform:
            sample['image'] = self.transform(sample['image'])
        
        return sample
    
    
data_transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize([224, 224]),
                transforms.RandomCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.471, 0.460, 0.455], [0.292, 0.291, 0.299])
            ])

root_dir = '/home/lvlab/Pytorch/car_data'
train_file = '/home/lvlab/Pytorch/car_data/train_list.txt'
test_file = '/home/lvlab/Pytorch/car_data/test_list.txt'
local_train = '/home/lvlab/Pytorch/train_vectors_car.npy'
local_test = '/home/lvlab/Pytorch/test_vectors_car.npy'

image_datasets = {'train': Bird_Dataset_Local(train_file, root_dir, local_train, data_transform),
                  'test': Bird_Dataset_Local(test_file, root_dir, local_test, data_transform)}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

print(image_datasets['train'][0]['image'].shape, image_datasets['test'][0]['image'].shape)
print(dataset_sizes)

torch.Size([3, 224, 224]) torch.Size([3, 224, 224])
{'test': 8041, 'train': 8144}


In [7]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, resume=None):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    start_epoch = 0
    if resume:
        ckpt = torch.load(resume)
        model = ckpt['best_model']
        optimizer = ckpt['optimizer']
        scheduler = ckpt['scheduler']       
        start_epoch = ckpt['epoch']
    
    for epoch in range(start_epoch, start_epoch + num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))
        print('-' * 10)
                
        # Each epoch has a training and validation phase
        for phase in ['test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            start_time = time.time()        
            if epoch % 5 == 0 and phase == 'train':
                for param_group in optimizer.param_groups:
                    lr = param_group['lr']
                    print("***********************")
                    print("learning rate = %f" % lr)
            
            # Iterate over data.
            for i_batch, sample_batched in enumerate(dataloaders[phase]):
                inputs = sample_batched['image']
                labels = sample_batched['label']
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                if i_batch % 50 == 0 and phase == 'train':
                    print("Iteration %d, loss = %f" % (i_batch, loss))

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'test':
                scheduler.step(epoch_loss)
                
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            cost_time = (time.time() - start_time)/60.0
            print('{} 1 epoch time: {:.2f}min'.format(
                phase, cost_time))
            
            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({
                    'epoch': epoch,
                    'test_loss': epoch_loss,
                    'test_acc': epoch_acc,
                    'optimizer': optimizer,
                    'scheduler': scheduler,
                    'best_model': model},
                    os.path.join('/home/lvlab/Pytorch/object_car/', '%03d.ckpt' % epoch))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
import copy
import time
import warnings
import torch.optim as optim
from torch.optim import lr_scheduler

warnings.filterwarnings('ignore')

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

local_net = models.vgg16(pretrained=True)

local_net.classifier = nn.Sequential(*list(local_net.classifier.children())[:1])
local_net.features[30] = nn.AvgPool2d(kernel_size=14, stride=1, padding=0)
local_net.features.add_module('31', nn.Dropout(0.8)) 
local_net.classifier[0] = nn.Linear(in_features=512, out_features=200, bias=True)

local_net = local_net.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
optimizer_ft = optim.SGD(local_net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)

exp_lr_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_ft, mode='min', factor=0.5, patience=10)

model_best = train_model(local_net, criterion, optimizer_ft,
                         exp_lr_scheduler, num_epochs=100)

In [9]:
torch.save(model_best, '/home/lvlab/Pytorch/object_car')